In [4]:
import pandas as pd
import altair as alt
import lib.utils.viz.KD as KD
folder="./Cifar10/ResNet101/normal"

In [5]:
import altair as alt
from lib.utils.viz import Feat


In [6]:
data=Feat.load_data(folder)

In [7]:
data.head()

,Unnamed: 0,test_acc,test_teacher/acc,test_loss,test_eval,train_acc,train_teacher/acc,train_loss,train_eval,epoch,...,teacher,distillation,last_layer,student_layer,teacher_layer,layer,transform,dataset,exp_name,student last_layer
0,0,88.77,93.94,0.015080,1.395208,94.672,99.886,0.009206,0.413442,50,...,ResNet101,att_max,"KD,T-8",6,26,0,"none,","cifar10,",las,"MobileNet,KD,T-8"
1,1,89.10,93.94,0.014359,1.279313,94.698,99.884,0.009395,0.416910,50,...,ResNet101,att_max,"KD,T-8",15,56,1,"none,","cifar10,",las,"MobileNet,KD,T-8"
2,2,89.04,93.94,0.015527,1.352884,94.832,99.908,0.009360,0.395154,50,...,ResNet101,att_max,"KD,T-8",26,219,2,"none,","cifar10,",las,"MobileNet,KD,T-8"
3,3,89.46,93.94,0.019441,1.298690,95.106,99.906,0.014023,0.385567,50,...,ResNet101,att_max,"KD,T-8",55,239,3,"none,","cifar10,",las,"MobileNet,KD,T-8"
4,4,89.50,93.94,0.015188,1.347961,94.724,99.880,0.009325,0.408336,50,...,ResNet101,att_mean,"KD,T-8",6,26,0,"none,","cifar10,",las,"MobileNet,KD,T-8"


In [12]:
detalle=['test_acc', 'test_loss', 'test_eval',
               'train_acc','train_loss', 'train_eval']


scale="log"
chart=alt.Chart(data).mark_point().encode(
    alt.X(alt.repeat("column"), 
          type='quantitative',
          scale=alt.Scale(zero=False,base=10,type=scale)),
    alt.Y(alt.repeat("row"), 
          type='quantitative',
          scale=alt.Scale(zero=False,base=10,type=scale)),
    color='student last_layer',
    shape='distillation'
).properties(
    width=200,
    height=200
).repeat(
    row=detalle,
    column=detalle
)

chart

alt.RepeatChart(...)

# Experimentos
En general se quiere probar que
1- Los algoritmos permiten aprender

2- Los algoritmos generalizan

3- Los algoritmos entrenan mas rápido que la wea cruda (Estabilidad, velocidad)

5- Cuanto depende del dataset ese aprendizaje

4- Son capaces de aprender de alguna distribución generada en la vecindad del dataset

6- Permiten una mejor estabilidad frente al ruido que el entrenamiento en crudo (En que momento pasa el umbral de tanto x)

7- Feats permiten una mejor estabilidad frente al ruido que el uso de logits solo

8- Funcionamiento de feats > funcionamiento de KD > funcionamiento de CE solo


Por cada experimento interesa

1- velocidad de convergencia (estabilidad?) (En que epoca el algoritmo pasa 3 epochs sin crecer)

2- Acc en test 

3- Acc en test / train

4- Mejores y peores

5- Delta sobre KD, CE

6- grafico que permita ver tendencia según tecnica/capa

7- media de feats por capa

Los experimentos disponibles sobre Cifar 10 son

| Experimento    | ce | kd | feat |   |
|----------------|----|----|------|---|
| Dataset Normal | x  | x  | x    |   |
| Ruido  (0,1-1) |    | x  | x    |   |
| GAN            |    | x  | x    |   |
| VAE            |    | x  | x    |   |

Podria faltar grid de ruido con temperatura
Podría ser interesante probar con alguna medida de distancia entre y_t y y_s, implica correr todo de nuevo

### Carga de datos

In [99]:
import pandas as pd
import altair as alt


CE_path=["Cifar10/%s/record.json"%model for model in ["ResNet101","ResNet18","MobileNet"]]

import json
def load_teachers(CE_path):
    def flatten_record(path):
        i=json.load(open(path,'r'))
        d={}
        for k,v in i.items():
            if isinstance(v, dict) :
                for l in v.values():
                    for ki,vi in l.items():
                        #print(l.values())
                        d["/".join([k,ki])]=vi
            else:
                d[k]=v
        return d

    d={'model':[],
     'test/acc': [],
     'test/loss ': [],
     'train/acc': [],
     'train/loss ': [],
     'epoch': [],
     'train_step': [],
     'test_step': []}

    for path in CE_path:
        d['model'].append(path.split("/")[-2])
        for k,v in flatten_record(path).items():
            d[k].append(v)
    return pd.DataFrame.from_dict(d)

data={}
data['teachers']=load_teachers(CE_path)
data["students"]={'feat':{},
                  'kd':{}}


import glob
paths=glob.glob("Cifar10/**/summary.csv",recursive=True)
print(paths)
for path in paths:
    ks=path.split("/")[-2].split("_")
    data["students"][ks[0]][ks[1]]=pd.read_csv(path)



['Cifar10/ResNet101/feat_VAE/summary.csv', 'Cifar10/ResNet101/feat_normal/summary.csv', 'Cifar10/ResNet101/kd_GAN/summary.csv', 'Cifar10/ResNet101/feat_CE/summary.csv', 'Cifar10/ResNet101/feat_GAN/summary.csv', 'Cifar10/ResNet101/kd_noise/summary.csv', 'Cifar10/ResNet101/kd_normal/summary.csv', 'Cifar10/ResNet101/kd_VAE/summary.csv', 'Cifar10/ResNet101/feat_noise/summary.csv']


In [172]:
data["students"]['teacher_CE']=data["teachers"][data["teachers"]['model']!="ResNet101"]

In [197]:
bar_source=data["students"]['teacher_CE']
bar_source['test_acc']=bar_source['test/acc']
bar_source['delta_test']=[i-data["teachers"].at[0,'test/acc'] for i in bar_source['test_acc']]
bar_source['test/train']=bar_source['test_acc']/bar_source["train/acc"]

bar_source['dist']='CE'
bar_source

/home/jpruiz/anaconda3/envs/Memoria1/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jpruiz/anaconda3/envs/Memoria1/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/jpruiz/anaconda3/envs/Memoria1/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

,model,test/acc,test/loss,train/acc,train/loss,epoch,train_step,test_step,test_acc,delta_test,test/train,dist
1,ResNet18,93.34,0.533314,99.846,0.004576,399,156400,40000,93.34,-0.6,0.93484,CE
2,MobileNet,89.94,0.580561,99.544,0.013739,399,156400,40000,89.94,-4.0,0.90352,CE


## Graficos Dataset Normal
### Tabla teachers, (mejores, peores, media) en KD y con feats

### Valores de KD contra temperatura

In [216]:
source=data['students']['kd']['normal']
source['temp']=[int(i.split('-')[-1]) for i in source['distillation']]
source['dist']=[i.split(',')[0] for i in source['distillation']]

source['delta_test']=source['test_acc']-source['test_teacher/acc']
source['test/train']=source['test_acc']/source['train_acc']

chart=alt.Chart(source,title="Diferencia en accuracy en entrenamiento con respecto a la red tutora.").mark_point(size=100).encode(
        alt.Y("delta_test", type='quantitative',title="Delta accuracy.",scale=alt.Scale(zero=False,base=10,type='linear')),
        alt.X("temp", type='quantitative',title="Temperatura (log).",scale=alt.Scale(zero=False,base=10,type='log')),
        color=alt.Color('student', legend=alt.Legend(title="Modelo")),
        shape=alt.Shape('dist', legend=alt.Legend(title="Tipo de destilacion")),
        #size=50
    ).interactive()

bar=alt.Chart(bar_source).mark_rule(opacity=0.5).encode(
                    y='delta_test',
                    color=alt.Color('model', legend=alt.Legend(title="Modelo")),
                    stroke=alt.Stroke('model', legend=alt.Legend(title="Modelo en Cross Entropy")),
                    size=alt.value(2))
d=bar+chart
d.properties(width=600,height=600).configure_axis(titleFontSize=12).configure_title(fontSize=15)


alt.LayerChart(...)

In [219]:
field="test/train"
chart=alt.Chart(source,title="Ratio  accuracy en validacion con accuracy en entrenamiento.").mark_point(size=100).encode(
        alt.Y(field, type='quantitative',title="Ratio de accuracy",scale=alt.Scale(zero=False,base=10,type='linear')),
        alt.X("temp", type='quantitative',title="Temperatura (log).",scale=alt.Scale(zero=False,base=10,type='log')),
        color=alt.Color('student', legend=alt.Legend(title="Modelo")),
        shape=alt.Shape('dist', legend=alt.Legend(title="Tipo de destilacion")),
        #size=50
    ).interactive()

bar=alt.Chart(bar_source).mark_rule(opacity=0.5).encode(
                    y=field,
                    color=alt.Color('model', legend=alt.Legend(title="Modelo")),
                    stroke=alt.Stroke('model', legend=alt.Legend(title="Modelo en Cross Entropy")),
                    size=alt.value(2))
d=bar+chart
d.properties(width=600,height=600).configure_axis(titleFontSize=12).configure_title(fontSize=15)


alt.LayerChart(...)

### Barras en feats
test_acc,*feats(layer), raya en logit (t8), en grid ([kd,ce,kdce] [mobilenet,resnet])

In [221]:
source=data['students']['feat']['normal']
source

,Unnamed: 0,test_acc,test_teacher/acc,test_loss,test_eval,train_acc,train_teacher/acc,train_loss,train_eval,epoch,...,student,teacher,distillation,last_layer,student_layer,teacher_layer,layer,transform,dataset,exp_name
0,0,88.77,93.94,0.015080,1.395208,94.672,99.886,0.009206,0.413442,50,...,MobileNet,ResNet101,att_max,"KD,T-8",6,26,0,"none,","cifar10,",las
1,1,89.10,93.94,0.014359,1.279313,94.698,99.884,0.009395,0.416910,50,...,MobileNet,ResNet101,att_max,"KD,T-8",15,56,1,"none,","cifar10,",las
2,2,89.04,93.94,0.015527,1.352884,94.832,99.908,0.009360,0.395154,50,...,MobileNet,ResNet101,att_max,"KD,T-8",26,219,2,"none,","cifar10,",las
3,3,89.46,93.94,0.019441,1.298690,95.106,99.906,0.014023,0.385567,50,...,MobileNet,ResNet101,att_max,"KD,T-8",55,239,3,"none,","cifar10,",las
4,4,89.50,93.94,0.015188,1.347961,94.724,99.880,0.009325,0.408336,50,...,MobileNet,ResNet101,att_mean,"KD,T-8",6,26,0,"none,","cifar10,",las
5,5,89.31,93.94,0.014697,1.320271,95.008,99.878,0.009293,0.387705,50,...,MobileNet,ResNet101,att_mean,"KD,T-8",15,56,1,"none,","cifar10,",las
6,6,88.59,93.94,0.015862,1.386573,94.924,99.912,0.010165,0.401062,50,...,MobileNet,ResNet101,att_mean,"KD,T-8",26,219,2,"none,","cifar10,",las
7,7,90.20,93.94,0.025879,1.214057,95.380,99.906,0.020955,0.333898,50,...,MobileNet,ResNet101,att_mean,"KD,T-8",55,239,3,"none,","cifar10,",las
8,8,89.50,93.94,70.395203,1.299240,95.136,99.914,67.800794,0.384486,50,...,MobileNet,ResNet101,hint,"KD,T-8",6,26,0,"none,","cifar10,",las
9,9,90.62,93.94,175.718310,1.133683,96.138,99.910,160.660284,0.283008,50,...,MobileNet,ResNet101,hint,"KD,T-8",15,56,1,"none,","cifar10,",las


3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd,ce,kdce] [mobilenet,resnet])

4- majamama tiempo de convergencia, acc (epochs y tiempo)
### Graficos GAN
Tabla teachers, (mejores, peores, media) en KD y con feats

2- Barras con test_acc,*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
3- Barras con test_acc/train_acc*feats(layer), raya en logit (t8), en grid ([kd] [mobilenet,resnet])
4- majamama tiempo de convergencia, acc (epochs y tiempo)